<h1> Transformers </h1>


In [ ]:
# We always start with a dataset to train on. Download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-27 08:55:08--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-09-27 08:55:09 (52.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
#Opening the file
with open ('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()
print("The length of the dataset is ", len(text))
print(text[:1000])  #First Thousand characters

The length of the dataset is  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger f

In [ ]:
#Generating the character vocabulary - list of unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))  #Printing the vocabulary List
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
#Creating a mapping from the strings to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]   #Encoder builds a list of integer from the string of characters
decode = lambda n: "".join(itos[i] for i in n) #Decoder builds a string from a list of integers

print("Encoding of Hello World!", encode("Hello World!"))
print("Decode back to string", decode(encode("Hello World!")))

Encoding of Hello World! [20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Decode back to string Hello World!


In [ ]:
#Encoding the entire dataset to a Tensor
import torch
data_tensor = torch.tensor(encode(text),dtype=torch.long)
print(data_tensor.shape, data_tensor.dtype )
print(data_tensor[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
#Splitting the data into train and validation sets
n = int((0.8)*len(data_tensor))
train_data = data_tensor[:n]
val_data = data_tensor[n:]

In [ ]:
#Creating a block size which defines the maximum character range that model considers to predict the next characetr.
#The model can take 1 to block size number of characters for its next prediction

block_size = 8
x = train_data[:block_size] # Input to the model
y = train_data [1:block_size+1] # Predictions for the next character

#Lets see an example
for c in range(block_size):
  context = x[:c+1]
  target_prediction = y[c]
  print(f"When the input is {context} the predicted next charcter is {target_prediction}")

When the input is tensor([18]) the predicted next charcter is 47
When the input is tensor([18, 47]) the predicted next charcter is 56
When the input is tensor([18, 47, 56]) the predicted next charcter is 57
When the input is tensor([18, 47, 56, 57]) the predicted next charcter is 58
When the input is tensor([18, 47, 56, 57, 58]) the predicted next charcter is 1
When the input is tensor([18, 47, 56, 57, 58,  1]) the predicted next charcter is 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]) the predicted next charcter is 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the predicted next charcter is 58


In [ ]:
#Batching the data
import torch

torch.manual_seed(1337)
batch_size = 4  #Number of independent sequences that will be processed in parallel
block_size = 8 # Context length for predictions.

def get_batch(split):
  #Generates multiple small batch of data (input x and output y)
  data = train_data if split == 'train' else val_data

  #Create a tensor with random numbers within the data length. The random numbers in tensors will be the start index for each batch.
  index = torch.randint(len(data) - block_size, (batch_size,))
  #randint returns a tensor with random integers generated between low(inclusive) and high (excluded) limits.
  #Here low is the default 0 and the high is length(data) - block_size.
  #The shape of the tensor is defined by size which is (batch_size,) i.e (4,)
  #O/P: index = tensor([217291,  74725,  60587,  27397])

  #Extracting the chracter blocks for the generated blocks (4 blocks in total).
  #torch.stack concatenates the 4 blocks of tensors in a new dimension
  #while torch.cat stacks each block into a single tensor with out a new dimension.
  x = torch.stack([data[i: i+block_size] for i in index])
  y = torch.stack([data[i+1:i+block_size+1] for i in index])
  return x,y

In [ ]:
#Printing the inputs and its corresponding targets.

xb,yb = get_batch("trian")
print("inputs:")
print(xb.shape)
print(xb)

print("-------")

print("targets:")
print(yb.shape)
print(yb)



inputs:
torch.Size([4, 8])
tensor([[43, 58,  0, 39, 52, 42,  1, 45],
        [ 1, 58, 46, 43,  1, 51, 43, 56],
        [39, 56, 42, 43, 52,  7, 46, 53],
        [47, 52, 45,  1, 42, 47, 57, 41]])
-------
targets:
torch.Size([4, 8])
tensor([[58,  0, 39, 52, 42,  1, 45, 47],
        [58, 46, 43,  1, 51, 43, 56, 43],
        [56, 42, 43, 52,  7, 46, 53, 59],
        [52, 45,  1, 42, 47, 57, 41, 53]])


In [ ]:
for i in range(batch_size):
  for j in range(block_size):
    print("If the character is ",xb[i,:j+1].tolist())
    print("The predicted target must be ",yb[i,j])
  print("------Next BLock--------")

If the character is  [43]
The predicted target must be  tensor(58)
If the character is  [43, 58]
The predicted target must be  tensor(0)
If the character is  [43, 58, 0]
The predicted target must be  tensor(39)
If the character is  [43, 58, 0, 39]
The predicted target must be  tensor(52)
If the character is  [43, 58, 0, 39, 52]
The predicted target must be  tensor(42)
If the character is  [43, 58, 0, 39, 52, 42]
The predicted target must be  tensor(1)
If the character is  [43, 58, 0, 39, 52, 42, 1]
The predicted target must be  tensor(45)
If the character is  [43, 58, 0, 39, 52, 42, 1, 45]
The predicted target must be  tensor(47)
------Next BLock--------
If the character is  [1]
The predicted target must be  tensor(58)
If the character is  [1, 58]
The predicted target must be  tensor(46)
If the character is  [1, 58, 46]
The predicted target must be  tensor(43)
If the character is  [1, 58, 46, 43]
The predicted target must be  tensor(1)
If the character is  [1, 58, 46, 43, 1]
The predic

<h2> Bigram Language Model </h2>



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
n_embed = 32
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    #self.token_embedding_table = nn.Embedding(vocab_size,vocab-size) # A simple lookup table that store word embeddings of fixed dictionary and size (A 65x65 lookup table)
    #Here the length of lookup table is 65 and each embedding 65 length long where 65 is the vocab_size
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed) # A lookup table for each character with its 32length embedding

    #We can also embed the positions of the characters - Positional Encoding
    self.position_embedding_table = nn.Embedding(block_size,n_embed)#Each position index is transformed into a 32 character embedding

    #A linear layer to transform the embeddings into logits
    self.languagemodel_head = nn.Linear(n_embed,vocab_size) #n_embed is the input layer size and the vocab_size is the output layer size

  def forward(self,inputs,targets = None):

    token_emb = self.token_embedding_table(inputs) # Each character is transformed into a 32 character length embedding
    #print(tok_emb) This is the first batch of eight characters where each character is a n_embed size embedding = (B,T,32) tensor eg.(32,8,32)

    position_emb = self.position_embedding_table(torch.arange(8))#Each position index is transformed into a 32 character embedding
    #print(position_emb) #The embedding of each position from 0 to 7 = (8,32) tensor

    #print(token_emb.shape, position_emb.shape)
    x = token_emb + position_emb  # Now the input has both the chracter and the positional imformation
    logits = self.languagemodel_head(x)

    #logits = self.token_embedding_table(inputs) # Logits is our prediction for the next character.
    #Each character is converted into a 65 length embedding. Logits is of shape (Batch,Block(Time),Channels) = (4,8,64)
    #inputs is a tensor of block of character indicies and not the character itself (4 batches of character blocks (8 characters))

    #Calculating Loss on our predictions (logits) using the Cross Entropy loss
    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      #The logits is a 3D tensor of shape (B,T,C) but cross_entropy loss function in pytorch expects the input to be a 2D tensor of shape (N,C) where N is number of batches and C is number of classes.
      # Here the classes are  each embedding for a character. So we reshape the logits to (B*T, C) = (32,65) and out target output to be (B*T) = (32)
      logits = logits.view(B*T, C) #view reshapes the tensor to the given shape
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)

    return logits,loss


#A simple bigram model to generate the next character (as many characters as max_new_tokens) based on its previous character.
  def generate(self, indices, max_new_tokens):
    for _ in range(max_new_tokens):
      #get the prediction for next character based on the inputs
      logits, loss = self(indices)
      # Logits is of shape (B,T,C) where we are interested in the time dimension ie. the previous element only
      logits = logits [:, -1, :] #Now the logits are of shape (B,C)
      #Apply softmax to get the probabilities from the logits (Log odds)
      prob = F.softmax(logits, dim = -1) # dim attribute tells the dimension across which the probability distribution is calculated

      #Find the index of the next character based on the above probability distribution
      next_char_idx = torch.multinomial(prob,max_new_tokens,replacement=True)
      #append this predicted character index to the previous predicted chracters until we reach max new tokens
      indicies = torch.cat((indices,next_char_idx),dim =1)
    return indicies





In [ ]:
m = BigramLanguageModel(vocab_size)
output, loss  = m(xb,yb) # xb is the inputs and the yb is the targets
print(output.shape)
print(loss)
inputs = torch.zeros((1,1),dtype = torch.long) # This is a 1x1 tensor with the index 0. As per the encoding vocab list it is a new line character.
max_new_tokens = 150
generated_tensor = m.generate(inputs,max_new_tokens) # This generates a tensor of (1,151) 1 batch of 151 character indicies.
#So we convert the tensor to a list and decode the character indicies to characters.
generated_sequence = decode(generated_tensor[0].tolist())
print(generated_sequence)


torch.Size([32, 65])
tensor(4.4019, grad_fn=<NllLossBackward0>)

pQRe'HEg?roHWrG!DG itDNERQESiRJmL3;LkfRmm':YRgvvvPH'Ed!lRLQa,j.Vk t,jagtUtS tvwDElEyHyZi'Nd3gVxxvEy!DVw!,A?Ry?VRRjVoRjNcCDlmaCptvLeC.vjMaMhyr
bOgzygXz


In [ ]:
#Creating a optimizer for the model
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) #The m.parameters() refer to the weights and biases (iteratable model parameters) that get updated while training.

In [ ]:
#Averaging the loss over multiple iterations and taking the average loss

@torch.no_grad()
#Disables gradient calculation even though the model inputs have requires_grad = True. This just to calculate the average loss and
#.backward() will not be called on these calcuations. Saves the memory consumption by avaoiding to store all the intermediate results.

def average_loss(iters):
  out = {}
  m.eval() # Switches the model to evaluation model where layers like dropout, BatchNorm do not have any effect
  for split in ['train', 'val']: #Calculate the loss of data in both train and validation batch
    losses = torch.zeros(iters) # Intializing the losses to zeros
    for k in range (iters):
      X,Y = get_batch(split)
      logits,loss = m(X,Y)    #Calculating the logits and loss for the batch of input and outputs
      losses[k] = loss.item() #Extracting the loss for kth iteration
    out[split] = losses.mean()
  m.train() # Switching the model back to training mode
  return out


In [ ]:
#Training the model

batch_size =32

for iterations in range (1000):

  #sample a batch of data
  xb,yb = get_batch("train") # We get a 32 batches of 8 character indicies. shape (32,8)

  #Generate the predicted character and caluclate the loss by comparing to the actual target character yb
  logits, loss = m(xb,yb) #This call the forward function.
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
  loss = average_loss(100)

  print(loss)



{'train': tensor(2.5892), 'val': tensor(2.6197)}
{'train': tensor(2.5847), 'val': tensor(2.6281)}
{'train': tensor(2.5854), 'val': tensor(2.6287)}
{'train': tensor(2.5926), 'val': tensor(2.6286)}
{'train': tensor(2.5897), 'val': tensor(2.6199)}
{'train': tensor(2.5916), 'val': tensor(2.6288)}
{'train': tensor(2.5993), 'val': tensor(2.6407)}
{'train': tensor(2.5840), 'val': tensor(2.6184)}
{'train': tensor(2.5653), 'val': tensor(2.6161)}
{'train': tensor(2.5793), 'val': tensor(2.6027)}
{'train': tensor(2.5818), 'val': tensor(2.6098)}
{'train': tensor(2.5808), 'val': tensor(2.6158)}
{'train': tensor(2.5736), 'val': tensor(2.6201)}
{'train': tensor(2.5737), 'val': tensor(2.6089)}
{'train': tensor(2.5836), 'val': tensor(2.6241)}
{'train': tensor(2.5681), 'val': tensor(2.6261)}
{'train': tensor(2.5960), 'val': tensor(2.6207)}
{'train': tensor(2.5828), 'val': tensor(2.6119)}
{'train': tensor(2.5829), 'val': tensor(2.6065)}
{'train': tensor(2.5889), 'val': tensor(2.6176)}
{'train': tensor(2.5

In [ ]:
#Generating a sequence with a slightly optimised model
inputs = torch.zeros((1,1),dtype = torch.long)
print(decode(m.generate(inputs,100)[0].tolist()))




E
SGNT
SSWTTFMWWTHY
Y
JAWB
SYSTBI
AA
HWT
TH
I
aEGTCW
BT

hB
S
JQATS
WWPHIS
SSBSTN
HHCL
T
EWSISESWPA


<h2> The mathematical trick in self -attention </h2>


In [ ]:
#Example of Naive Implementation of Averaging the past information (characters) to predict future character

#Creating a dummy x dataset
B,T,C = 4,8,2 #Batch, Time, Channels
x = torch.randn((B,T,C))
print("The inputs in the 1st batch \n",x[0,:,:])

#Averaging over the time dimension in each channel in each batch.
xbow = torch.zeros((B,T,C))
for b in range(B): #Iterating over each batch
  for t in range(T):
    x_previous = x[b,:t+1] # All characters in batch b from the 0th time index (past) to the current time index
    #print(x_previous.shape) x_previous is of shape (t,C) where t is current time index in the for loop.
    xbow[b,t] = torch.mean(x_previous,0) # 0 axis represents the averaging over the time axis
    #print(torch.mean(x_previous,1)) For the curious what does averaging over the channels (dim =1) look like !
print("The averaged x_bag_of_words for the 1st batch \n", xbow[0,:,:])



The inputs in the 1st batch 
 tensor([[-0.3376, -1.5154],
        [-0.8523, -0.3037],
        [-0.7137, -0.8800],
        [ 0.1575, -0.6906],
        [-0.7282,  1.1125],
        [-1.0222,  0.2645],
        [-0.2275,  0.3516],
        [ 0.8941, -1.3865]])
The averaged x_bag_of_words for the 1st batch 
 tensor([[-0.3376, -1.5154],
        [-0.5950, -0.9095],
        [-0.6345, -0.8997],
        [-0.4365, -0.8474],
        [-0.4948, -0.4554],
        [-0.5827, -0.3354],
        [-0.5320, -0.2373],
        [-0.3537, -0.3809]])


In [ ]:
#More efficient implementation of the averaging with Matrix Multiplication
'''Matrix Multiplication with a lower triangle matrix [[1,0,0],
                                                     [1,1,0],
                                                     [1,1,1]] with a input vector results in the summation of the past elemnets in time dimension
While normalising the lower triangular matrix along the rows such that each row sum up to 1 results in the averaging over the past elements in the time dimension
 weights = [[1,0,0],
            [0.5,0.5,0],
            [0.33,0.33,0.33]]'''

weights = torch.tril(torch.ones(T,T)) # A lower trianglular matrix
weights = weights/weights.sum(dim = 1,keepdim=True) #Summing along the rows and keep dims = True to preserve the dimension
print("The weights are \n",weights)

#Matrix Multiplication to calculate the average
xbow2 = weights @ x         #(T,T) @ (B,T,C) ==> (B,T,T) @ (B,T,C) pytorch adds a Batch dimension to weights by copying the same TxT weights acroos the batches.
print(xbow2[0])             #xbow2 is of shape (B,T,C)
print(torch.allclose(xbow,xbow2)) #True Tells us that xbow and xbow2 obtained through naive and mat multiplication give same results



The weights are 
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[-0.3376, -1.5154],
        [-0.5950, -0.9095],
        [-0.6345, -0.8997],
        [-0.4365, -0.8474],
        [-0.4948, -0.4554],
        [-0.5827, -0.3354],
        [-0.5320, -0.2373],
        [-0.3537, -0.3809]])
True


<h3>Normalising using the softmax function </h3>

Softmax function = $\frac{e^x}{\sum e^x}$

On applying the softmax function over the rows (dim =1):

In the first row, the one zero $e^0 = 1$ and the other -inf results in 0. Thus it results in $\frac{1}{1+0+..+0} = 1$

In the second row, there are two zeros and it results in $\frac{1}{1+1+0..0} = \frac{1}{2}$

And similarly in third row $\frac{1}{3}$ and so on till the eight row.

<h3> Why we use Softmax function for normalising rather than the previous version? </h3>

We may feel the previous version of creating a lower triangular matrix with ones and normalising them was a more intutive approach.

But when we train the model we may consider the weights as an affinity between the characetrs where some characters might find the another character next to it more interesting. This affinity is calculated based on the past characters only and this is why mask the future chracter indices with '-inf' at each time step. Thus these affinity modelled as weights are intialized to zero at the begining and are then trained to be updated based on the data fed to the model.




In [ ]:
#Implementation with Softmax function

lwr_triangle_mat = torch.tril(torch.ones(T,T)) #Creating a lower triangular matrix
weights = torch.zeros(T,T) # Intiating the weights to zeros
weights = weights.masked_fill(lwr_triangle_mat == 0, float('-inf'))
print("The initial weight tensor ",weights) # This for a lower traingle matrix where 1s are replaced by 0 and the 0s are replaced by -inf
weights = F.softmax(weights, dim = 1) # Normalising using the softmax function
print("The normalised weights ",weights)
#Aggregating over thee inputs with matrix multiplication
xbow3 = weights @ x
print("The aggregated first batch of bag of words",xbow[0])
torch.allclose(xbow,xbow3)


The initial weight tensor  tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
The normalised weights  tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.125

True

In [ ]:
# Single Self Attention Block
torch.manual_seed(1337)
B,T,C = 4,8,32 # Batch, Time, Channel dimension Channel is each character is encoded into a 32 length embedding
x = torch.randn((B,T,C)) # Random input

head_size =16  # HyperParameter
#For each input we create 3 vectors - Query, Key and a Value
Query = nn.Linear(C,head_size,bias=False) # A linear layer that creates a length 16 query vector for each input character
Key = nn.Linear(C,head_size,bias=False)   #A linear layer that creates a length 16 key vector for each input character
Value = nn.Linear(C, head_size,bias=False) #A linear layer that creates a length 16 value vector for each input character

q = Query(x) #q => (B,T,16)
k = Key(x)   #k => (B,T,16)
v = Value(x) #V => (B,T,16)

#Now the weights is what each character is looking for another character that it has most affinity to.
#It is a dot product of queries and keys of each cahracter.
weights = q @ k.transpose(2,1) * head_size ** (-0.5) # We can multiply the B batches of matrices of size (T,16) @ (T,16) so we transpose the key tensor.
#We only have to transpose the last two dimensions and not the Batch dimension
#print(weights) #(B,T,16) @ (B,16,T) => (B,T,T) Here T is 8.
#These weights represent the affinity that a character at a postion might have to another character. If the wieghts are higher indicates higher affinity.
# The multiplication with the 1/sqrt(head_size) results in diffused softmax rather than a sharp softmax similar to a one hot vector.

#Now we normalise these weights using masking and the softmax function.
lower_triangular_mask = torch.tril(torch.ones(T,T))
weights = weights.masked_fill(lower_triangular_mask ==0 , float('-inf'))
weights = F.softmax(weights,dim = 2)
#print(weights) #Normalised weights

out = weights @ v # We multiply it with the values of the x rather than the raw x (inputs) => (T,T) @(B,T,C) => (8,8) @ (4,8,16) =>(4,8,16)
print(out) # shape of out(4,8,16)





tensor([[[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007,
          -0.5239, -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,
           0.2862,  0.5710],
         [ 0.2507,  0.1815, -0.0388, -0.2458, -0.1356,  0.2369, -0.1588,
          -0.3209, -0.4772,  0.4530,  0.4388, -0.3604, -0.0859, -0.0803,
           0.1115,  0.9138],
         [ 0.3288,  0.0950, -0.1875, -0.0916, -0.0079,  0.0883, -0.0678,
          -0.1830, -0.4008,  0.0761,  0.3542, -0.1453, -0.1970, -0.0976,
           0.0109,  1.0278],
         [ 0.6067, -0.4271, -0.2246,  0.2273, -0.1100, -0.2183, -0.3709,
          -0.1340, -0.1130,  0.6494,  0.6441, -0.1387,  0.2489,  0.2713,
          -0.0351,  1.2031],
         [ 0.2010,  0.8507,  0.6533,  0.2228,  0.3173,  0.8365,  0.6526,
           0.3822, -0.6315, -1.2205, -0.4374, -0.2859, -0.9985,  0.1108,
          -0.1001,  0.5346],
         [ 0.1453,  0.4755,  0.1447, -0.2496, -0.0209,  0.4674,  0.0808,
          -0.2074, -0.5866,  0.0157,  0.1711, -0.374

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

#Hyperparameters
batch_size = 32
block_size = 256
n_embed = 128
n_layers = 4
n_head = 4

learning_rate = 1e-3
max_iter = 1500
eval_iters = 100
dropout =0.2
eval_interval = 500

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)


# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

@torch.no_grad()
def average_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

#Single Head attention
class Head(nn.Module):

  def __init__(self,head_size):
    super().__init__()
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
    self.head_size = head_size
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    B,T,C = x.shape    #Batch, Time, Channel (batch_size,block_size,n_embed)
    q = self.query(x)
    k = self.key(x)
    v = self.value(x)
    # All the above q,k,v are of shape (B,T,C)

    #Calculating the weights that are the affinities which is represented by the product between query and key.
    weights = q @ k.transpose(-2,-1)    #(B,T,C) @ (B,C,T)  === (B,T,T)
    #Normalizing and scaling weights using the softmax function
    weights = weights.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
    weights = F.softmax(weights,dim = -1)     # Normalisation along the last dimension

    weights = weights * self.head_size ** (-0.5)   # Scaling the weights to avoid sharpening at one character/token
    weights = self.dropout(weights)
    output = weights @ v
    #print(output.shape)
    return output

class MultiHeadAttention(nn.Module):
  #Multiple heads in parallel
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # Create a list of heads as many as the num_heads
    self.proj = nn.Linear(head_size * num_heads,n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
   #print("self.proj(out) ",out)
    out = self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  def __init__(self,n_embed):
    super().__init__()
    self.net = nn.Sequential(
                nn.Linear(n_embed, 4*n_embed),
                nn.ReLU(),
                nn.Linear(4*n_embed,n_embed),
                nn.Dropout(dropout)
    )

  def forward(self,x):
    return self.net(x)

class Block(nn.Module):

  #A transformer Block : Communication followed by Computation

  def __init__(self,n_heads,n_embed):
    super().__init__()
    self.LayerNorm1 = nn.LayerNorm(n_embed)
    head_size = n_embed//n_heads
    self.self_attention = MultiHeadAttention(n_heads,head_size)
    self.LayerNorm2 = nn.LayerNorm(n_embed)
    self.feedforward = FeedForward(n_embed)

  def forward(self,x):
    x = x + self.self_attention(self.LayerNorm1(x)) #Adding x directly creates the residual connections
    x = x + self.feedforward(self.LayerNorm2(x))
    return x


#GPT Language Model

class GPTLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size,n_embed)

    #self.selfatttention_head = Head(n_embed)          #Intializing the self attention head with head_size = n_embeds in case of a single head
    #

    self.blocks = nn.Sequential(*[Block(n_head,n_embed) for _ in range(n_layers)])
    #The asterisk * unpacks the list generated by the list comprehension into separate arguments.
    #This is often used when passing multiple arguments to a function or a class constructor that takes variable-length arguments (e.g., *args).

    self.LayerNorm3 = nn.LayerNorm(n_embed)
    self.languagemodel_head = nn.Linear(n_embed,vocab_size)


  def forward(self, indices, targets= None):
    #print(indices.shape)
    B,T = indices.shape

    #Performing Forward Pass
    token_embed = self.token_embedding_table(indices)
    position_embed = self.position_embedding_table(torch.arange(T))
    x = token_embed + position_embed
    #x = self.selfatttention_head(x)              # For single head - Calling the self attention head on inputs. The returned x will have token of x that has high attention compared to others.

    x = self.blocks(x)
    x = self.LayerNorm3(x)
    logits = self.languagemodel_head(x)

    #Calculating Loss
    if targets == None:
      loss = None

    else:
      B,T,C = logits.shape
      #Reshaping according to the cross-entropy loss function
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy (logits, targets)

    return logits, loss

  def generate(self, indices, max_new_tokens):
    for _ in range(max_new_tokens):
      #print("indices",indices)
      index_cropped = indices[:,-block_size:]
      #print("index_cropped ",index_cropped)
      logits,loss = self(index_cropped) # logits => (B,T,C)
      logits = logits[:,-1,:]     #(B,C)
      probability = F.softmax(logits,dim = -1) # Softmax along the channels
      next_index = torch.multinomial(probability,1,replacement=True) #(B,1)
      indices = torch.cat((indices,next_index),dim=1) #(B,T+1)
    return indices


model = GPTLanguageModel()

#Inspecting model parameters
print(sum(p.numel() for p in model.parameters())/1e6, "M Parameters")

#Optimizer for the model
optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate)

for iter in range(max_iter):
  if iter % eval_iters == 0 or iter == max_iter-1:
    losses = average_loss()
    print(f'step{iter}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}')

  #Get the batch data
  xb,yb = get_batch('train')

  #Evaluate Losses
  logits,loss = model(xb,yb)
  optimizer.zero_grad(set_to_none = True)

  #Calculate the gradients
  loss.backward()
  #Update the weights
  optimizer.step()

#generate from the model
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))






0.841281 M Parameters
step0: train loss 4.2345, val loss 4.2418
step100: train loss 2.5153, val loss 2.5178
step200: train loss 2.4748, val loss 2.4900
step300: train loss 2.4498, val loss 2.4657
step400: train loss 2.3777, val loss 2.4108
step500: train loss 2.2869, val loss 2.3186
step600: train loss 2.1836, val loss 2.2222
step700: train loss 2.1062, val loss 2.1521
step800: train loss 2.0351, val loss 2.1114
step900: train loss 1.9685, val loss 2.0613
step1000: train loss 1.9105, val loss 2.0151
step1100: train loss 1.8559, val loss 1.9828
step1200: train loss 1.8129, val loss 1.9477
step1300: train loss 1.7803, val loss 1.9193
step1400: train loss 1.7417, val loss 1.8993
step1499: train loss 1.7134, val loss 1.8632

Wouded Her-butind my a hesepcrounione
My to ter dote me.

My ROGHOMNIANDIANIUS:
Hard my lordo your dive ome, le weed stain.
God friasts blowicess it.
I as pontly of dow en if is brer's ful haven are:
Tahe houst that comet; you annot inquisong of clower.

DUCKE OF You w

<h3> About AutoGrad - Automatic differentiation </h3>


<h4> torch.no_grad </h4>
